In [ ]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing

In [ ]:
data = pd.read_csv("./bream_bay.csv")
data.head(2)

,DATE,TIME,Temperature,Day
0,1979-01-01,00:00:00,282.88965,0
1,1979-01-01,01:00:00,283.02603,0


In [ ]:
data["DATE_TIME"] = pd.to_datetime(data.DATE  + " " + data["TIME"])

In [ ]:
data.head(5)

,DATE,TIME,Temperature,Day,DATE_TIME
0,1979-01-01,00:00:00,282.88965,0,1979-01-01 00:00:00
1,1979-01-01,01:00:00,283.02603,0,1979-01-01 01:00:00
2,1979-01-01,02:00:00,283.13562,0,1979-01-01 02:00:00
3,1979-01-01,03:00:00,282.95294,0,1979-01-01 03:00:00
4,1979-01-01,04:00:00,282.92370,0,1979-01-01 04:00:00


In [ ]:
max_temp = data.Temperature.max()
max_temp

295.04376

In [ ]:
#data.Temperature = data.Temperature/max_temp

In [ ]:
for i in range(1,6):
    data["T_" + str(i)] = data.Temperature.shift(i)

In [ ]:
data.fillna(0.00,inplace=True)
data.head(10)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
0,1979-01-01,00:00:00,282.88965,0,1979-01-01 00:00:00,0.00000,0.00000,0.00000,0.00000,0.00000
1,1979-01-01,01:00:00,283.02603,0,1979-01-01 01:00:00,282.88965,0.00000,0.00000,0.00000,0.00000
2,1979-01-01,02:00:00,283.13562,0,1979-01-01 02:00:00,283.02603,282.88965,0.00000,0.00000,0.00000
3,1979-01-01,03:00:00,282.95294,0,1979-01-01 03:00:00,283.13562,283.02603,282.88965,0.00000,0.00000
4,1979-01-01,04:00:00,282.92370,0,1979-01-01 04:00:00,282.95294,283.13562,283.02603,282.88965,0.00000
5,1979-01-01,05:00:00,282.90910,1,1979-01-01 05:00:00,282.92370,282.95294,283.13562,283.02603,282.88965
6,1979-01-01,06:00:00,282.45352,1,1979-01-01 06:00:00,282.90910,282.92370,282.95294,283.13562,283.02603
7,1979-01-01,07:00:00,282.10270,1,1979-01-01 07:00:00,282.45352,282.90910,282.92370,282.95294,283.13562
8,1979-01-01,08:00:00,282.19528,1,1979-01-01 08:00:00,282.10270,282.45352,282.90910,282.92370,282.95294
9,1979-01-01,09:00:00,282.22452,1,1979-01-01 09:00:00,282.19528,282.10270,282.45352,282.90910,282.92370


In [ ]:
#clean_data = data[['Day', 'Cloud','Humidity','T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]
clean_data = data[['Day','T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]

In [ ]:
clean_data.head(2)

,Day,T_1,T_2,T_3,T_4,T_5,Temperature
0,0,0.00000,0.0,0.0,0.0,0.0,282.88965
1,0,282.88965,0.0,0.0,0.0,0.0,283.02603


In [ ]:
training_data = data[data.DATE_TIME < pd.to_datetime("01/01/2015")]

In [ ]:
training_data.tail(3)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
315573,2014-12-31,21:00:00,283.14084,0,2014-12-31 21:00:00,283.18637,283.37387,283.64978,283.70068,283.81320
315574,2014-12-31,22:00:00,282.21133,0,2014-12-31 22:00:00,283.14084,283.18637,283.37387,283.64978,283.70068
315575,2014-12-31,23:00:00,282.23007,0,2014-12-31 23:00:00,282.21133,283.14084,283.18637,283.37387,283.64978


In [ ]:
val_data = data[(data.DATE_TIME >= pd.to_datetime("01/01/2015")) & (data.DATE_TIME < pd.to_datetime("01/01/2017"))]

In [ ]:
test_data = data[data.DATE_TIME >= pd.to_datetime("01/01/2017")]

In [ ]:
test_data.tail(3)

,DATE,TIME,Temperature,Day,DATE_TIME,T_1,T_2,T_3,T_4,T_5
350637,2018-12-31,21:00:00,287.28946,0,2018-12-31 21:00:00,287.32630,287.30786,287.46216,287.78230,288.21298
350638,2018-12-31,22:00:00,286.60776,0,2018-12-31 22:00:00,287.28946,287.32630,287.30786,287.46216,287.78230
350639,2018-12-31,23:00:00,286.05734,0,2018-12-31 23:00:00,286.60776,287.28946,287.32630,287.30786,287.46216


In [ ]:
clean_train = training_data[['Day','T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]
clean_test = test_data[['Day','T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]
clean_val = val_data[['Day','T_1','T_2', 'T_3', 'T_4', 'T_5','Temperature']]

In [ ]:
X_train,y_train = clean_train.drop(["Temperature"],axis=1),clean_train.Temperature
X_test,y_test = clean_test.drop(["Temperature"],axis=1),clean_test.Temperature
X_val,y_val = clean_val.drop(["Temperature"],axis=1),clean_val.Temperature

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_val = np.reshape(X_val, (X_val.shape[0],X_val.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [ ]:
n_layer = 12
n_lag = 6

In [ ]:
model = Sequential()
model.add(LSTM(n_layer, activation = 'relu', input_shape = (n_lag,1)))
#model.add(LSTM(128))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12)                672       
                                                                 
 dense (Dense)               (None, 1)                 13        
                                                                 
Total params: 685
Trainable params: 685
Non-trainable params: 0
_________________________________________________________________


In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
history = model.fit(
    X_train, y_train,
    batch_size=1024,
    epochs=100,
    validation_data=(X_val, y_val),
    callbacks=[tensorboard_callback]
)

Epoch 1/100
309/309 [==============================] - 9s 19ms/step - loss: 409.9699 - val_loss: 0.3980
Epoch 2/100
309/309 [==============================] - 5s 16ms/step - loss: 0.8400 - val_loss: 0.3970
Epoch 3/100
309/309 [==============================] - 6s 19ms/step - loss: 0.7641 - val_loss: 0.3953
Epoch 4/100
309/309 [==============================] - 5s 16ms/step - loss: 0.7605 - val_loss: 0.3937
Epoch 5/100
309/309 [==============================] - 5s 17ms/step - loss: 0.7611 - val_loss: 0.3987
Epoch 6/100
309/309 [==============================] - 5s 16ms/step - loss: 0.7765 - val_loss: 0.3934
Epoch 7/100
309/309 [==============================] - 5s 16ms/step - loss: 0.7285 - val_loss: 0.3661
Epoch 8/100
309/309 [==============================] - 4s 11ms/step - loss: 0.6882 - val_loss: 0.3533
Epoch 9/100
309/309 [==============================] - 3s 10ms/step - loss: 0.6720 - val_loss: 0.3407
Epoch 10/100
309/309 [==============================] - 3s 10ms/step - loss: 0.6

In [ ]:
pred = model.predict(X_test)

In [ ]:
pred

array([[281.1946 ],
       [281.5707 ],
       [281.77386],
       ...,
       [286.99042],
       [287.00592],
       [286.7102 ]], dtype=float32)

In [ ]:
preddata = pd.DataFrame(data=pred[0:,0:])

In [ ]:
preddata['pred'] = preddata[0]

In [ ]:
preddata

,0,pred
0,281.194611,281.194611
1,281.570709,281.570709
2,281.773865,281.773865
3,281.744568,281.744568
4,281.777069,281.777069
...,...,...
17515,287.269989,287.269989
17516,287.049133,287.049133
17517,286.990417,286.990417
17518,287.005920,287.005920


In [ ]:
clean_test['Predicted'] = preddata['pred'].values
test_data['Predicted'] = preddata['pred'].values

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
clean_data2 = test_data[['DATE_TIME','Temperature','Predicted']]

In [ ]:
from sklearn.metrics import mean_squared_error
rms = mean_squared_error(clean_data2.Temperature, clean_data2.Predicted, squared=False)

In [ ]:
rms

0.5180077915086204

In [ ]:
#clean_data2.Temperature = clean_data2.Temperature * max_temp
#clean_data2.Predicted = clean_data2.Predicted * max_temp

In [ ]:
clean_data2

,DATE_TIME,Temperature,Predicted
333120,2017-01-01 00:00:00,281.61188,281.194611
333121,2017-01-01 01:00:00,281.79382,281.570709
333122,2017-01-01 02:00:00,281.85373,281.773865
333123,2017-01-01 03:00:00,281.91360,281.744568
333124,2017-01-01 04:00:00,282.24295,281.777069
...,...,...,...
350635,2018-12-31 19:00:00,287.30786,287.269989
350636,2018-12-31 20:00:00,287.32630,287.049133
350637,2018-12-31 21:00:00,287.28946,286.990417
350638,2018-12-31 22:00:00,286.60776,287.005920


In [ ]:
#clean_data2.to_csv("Location20_LSTM.csv", index = False)